# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***

## Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

## Read in an example image

In [ ]:
#Reads in the image to 'image'
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#Printing out some stats and plotting image
print('Image dimensions:', image.shape)
plt.imshow(image)

## Define functions for image processing

In [ ]:
def grayscale(img):
    """Applies the Grayscale transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    Only keeps the region of the image defined by the polygon
    formed from 'vertices'. The rest of the image is set to black.
    """
    #Defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #Filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #Returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=4):
    """
    This function draws 'lines' onto 'img' with 'color' and 'thickness'.    
    Lines are drawn on the image inplace (mutates the image).
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """Returns an image with Hough lines drawn."""
    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    """Use following code for outputting raw lane lines:"""
    #line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)
    #return line_img
    
    """For full lane lines, return lines for extrapolating function"""
    return lines
    
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    Adds "img" to "initial image"
    The result image is computed as follows:
    initial_img * α + img * β + γ
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def extrapolate_lines(lines):
    """
    Extrapolates lines from Hough transform to full lane lines.
    Code on solution from Matt Hardwick on Medium
    Expects lines, not a full image!
    When sorting according to the slope, we have to take into account that
    the y axis grows in the opposite direction than in a conventional coordinate system.
    Therefore:
    Positive slope -> right line
    Negative slope -> right line
    """
    
    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            
            if math.fabs(slope) < 0.5: # <-- Only consider extreme slope
                continue

            if slope <= 0:
                left_line_x.extend([x1, x2])
                left_line_y.extend([y1, y2])
            else:
                right_line_x.extend([x1, x2])
                right_line_y.extend([y1, y2])
                
    #Set fixed value for start and end point on y axis"
    min_y = 320 # <-- Just below the horizon
    max_y = 540 # <-- The bottom of the image
    
    #Linear fitting using the grouped Hough lines, returns an object that represents x = m*y + b
    fit_left = np.poly1d(np.polyfit(
        left_line_y,
        left_line_x,
        deg=1
    ))
    
    fit_right = np.poly1d(np.polyfit(
        right_line_y,
        right_line_x,
        deg=1
    ))
    
    #Find start and end "x" values using linear equation
    
    left_x0 = int(fit_left(max_y))
    left_x1 = int(fit_left(min_y))
    
    right_x0 = int(fit_right(max_y))
    right_x1 = int(fit_right(min_y))
    
    #Draw lines onto a black image
    
    full_lines = np.zeros((540, 960, 3), dtype=np.uint8)
    draw_lines(
        full_lines,
        [[
            [left_x0, max_y, left_x1, min_y],
            [right_x0, max_y, right_x1, min_y],
        ]],
            thickness=8,
    )

    return full_lines

## Pipeline

In [ ]:
def process_image(image):
    """
    Complete pipeline for processing images.
    Takes raw colored images, outputs the image with lane lines indicated with straight red lines
    Contains commented imshow functions for each step for debugging / image plotting purposes.
    Raw lane lines can also be generated by uncommenting the relevant sections.
    (In this case however, the hough lines function also needs to be modified.)
    """

    gray = grayscale(image)
    #plt.imshow(gray, cmap='gray')
           
    blurred = gaussian_blur(gray, 3)
    #plt.imshow(blurred, cmap='gray')
           
    edges = canny(blurred, 50, 150)
    #plt.imshow(edges, cmap='gray')

    vertices = np.array([[(90,540) , (450, 325), (520, 325), (940,540)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    #plt.imshow(masked_edges, cmap='gray')

    raw_lane_lines = hough_lines(masked_edges, 4, np.pi/60, 60, 100, 100)
    #plt.imshow(raw_lane_lines, cmap='gray')
    
    full_lane_lines = extrapolate_lines(raw_lane_lines)
    #plt.imshow(full_lane_lines, cmap='gray')

    #result_raw = weighted_img(raw_lane_lines, image, α=0.8, β=1., γ=0.)
    #plt.imshow(result_raw)
    
    result_full = weighted_img(full_lane_lines, image, α=0.8, β=1., γ=0.)
    #plt.imshow(result_full, cmap='gray')
    
    #return result_raw
    return result_full

## Pipeline applied to the example image

In [ ]:
annotated_image = process_image(image)
plt.imshow(annotated_image, cmap='gray')

## Pipeline applied to the videos
Code uses fl_image function of VideoClip class to apply pipeline to the test videos.

Output is the annotated video with lane lines indicated by straight red lines.

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,3) <--- For testing
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
#clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,3) <--- For testing
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))